In [1]:
import os
import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

In [2]:
detect_fn = tf.saved_model.load('saved_model')

In [6]:
def boundingbox(im,model,threshold=0.3):
    image_np=np.array(im)
    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = model(input_tensor)
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    boxes=list()
    height=image_np.shape[0]
    width=image_np.shape[1]
    for i in range(detections['detection_boxes'].shape[0]):
        box=detections['detection_boxes'][i]
        if detections['detection_scores'][i]>threshold:
            ymin=int(box[0]*height)
            xmin=int(box[1]*width)
            ymax=int(box[2]*height)
            xmax=int(box[3]*width)
            boxes.append([detections['detection_scores'][i],xmin,ymin,xmax,ymax])
    return boxes

In [7]:
def write_to_txt(suffix,boxes,output):
    with open(output+suffix+'.txt', 'w') as f:
        for box in boxes:
            f.write('Table')
            f.write(' ')
            for item in box:
                f.write(str(item))
                f.write(' ')
            f.write('\n')

In [9]:
path = 'Test/'
output='Test detections/'
for f in os.listdir(path):
    suffix = f.split('.')[0]
    im = Image.open(path+f)
    im = im.convert("RGB")
    boxes=boundingbox(im,detect_fn)
    write_to_txt(suffix,boxes,output)

In [13]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

In [14]:
category_index = label_map_util.create_category_index_from_labelmap('label_map.pbtxt', use_display_name=True)

In [16]:
path='Test/'
output='Test images/'
for f in os.listdir(path):
    im = Image.open(path+f)
    im = im.convert("RGB")
    image_np=np.array(im)
    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = detect_fn(input_tensor)
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)
    cv.imwrite(output+f,image_np)